In [1]:
import pandas as pd
import logging
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random
import math
import time

### Assignment Owner: Tian Wang

#######################################
####Q2.1: Normalization

In [2]:
data = np.loadtxt(open("hw1-data.csv","rb"),delimiter=",",skiprows=1)

In [52]:
def split_train_test(data):
    int(data.shape[0])
    int(data.shape[1])
    train = data[0:1,:]
    test = data[1:2,:]
    for i in range(2,int(data.shape[0])):
        if random.random() < 0.5:
            train = np.concatenate((train,data[i:i+1,:]))
        else :
            test = np.concatenate((test,data[i:i+1,:]))
    return (train,test)

In [5]:
train,test=split_train_test(data)

In [53]:
def feature_normalization(train, test):
    """Rescale the data so that each feature in the training set is in
    the interval [0,1], and apply the same transformations to the test
    set, using the statistics computed on the training set.

    Args:
        train - training set, a 2D numpy array of size (num_instances, num_features)
        test  - test set, a 2D numpy array of size (num_instances, num_features)
    Returns:
        train_normalized - training set after normalization
        test_normalized  - test set after normalization

    """
    # TODO
    parms = {"min":[],"range":[]}

    for i in range(0,int(train.shape[1])):
        if min(train[:,i]) == 0 and max(train[:,i]) == 0:
            parms['min'].append(0)
            parms['range'].append(0)
            continue
        else:
            parms['min'].append(min(train[:,i]))
            parms['range'].append(max(train[:,i])-min(train[:,i]))


    for i in range(0,int(test.shape[1])):
        if i ==0:
            if parms["range"][i]==0:
                test_normalized = (test[:,i]-parms["min"][i])
            else:
                test_normalized = (test[:,i]-parms["min"][i])/parms["range"][i]
        else:
            if parms["range"][i]==0:
                test_normalized = np.vstack((test_normalized,(test[:,i]-parms["min"][i]))) 
            else:
                test_normalized = np.vstack((test_normalized,((test[:,i]-parms["min"][i])/parms["range"][i])))
    test_normalized = np.transpose(test_normalized)


                
    for i in range(0,int(train.shape[1])):
        if i ==0:
            if parms["range"][i]==0:
                train_normalized = (train[:,i]-parms["min"][i])
            else:
                train_normalized = (train[:,i]-parms["min"][i])/parms["range"][i]
        else:
            if parms["range"][i]==0:
                train_normalized = np.vstack((train_normalized,(train[:,i]-parms["min"][i]))) 
            else:
                train_normalized = np.vstack((train_normalized,((train[:,i]-parms["min"][i])/parms["range"][i])))
    train_normalized = np.transpose(train_normalized)
    
    return (train_normalized,test_normalized)


In [7]:
train_norm,test_norm=feature_normalization(train, test)
theta = np.expand_dims(np.ones(int(train_norm.shape[1])), axis=1)
#print a,b

########################################
####Q2.2a: The square loss function

In [54]:
def compute_square_loss(X, y, theta):
    """
    Given a set of X, y, theta, compute the square loss for predicting y with X*theta
    
    Args:
        X - the feature vector, 2D numpy array of size (num_instances, num_features)
        y - the label vector, 1D numpy array of size (num_instances)
        theta - the parameter vector, 1D array of size (num_features)
    
    Returns:
        loss - the square loss, scalar
    """
    loss = 0 #initialize the square_loss
    #TODO
    
    part = np.dot(X,theta)-y
    loss = loss + np.dot(np.transpose(part),part)
    loss = loss/(2*int(y.shape[0]))
    
    return loss

In [44]:
np.dot(X,theta)

array([ 1191.91,  1187.48,  1174.24,  1175.96,  1166.48,  1154.23,
        1137.73,  1146.97,  1138.78,  1140.55,  1161.68,  1166.7 ,
        1169.82,  1163.83,  1159.71,  1159.32,  1152.92,  1156.33,
        1168.97,  1182.11,  1188.41,  1191.55,  1194.55,  1192.87,
        1201.64,  1215.87,  1217.87,  1215.73,  1196.81,  1201.67,
        1193.15,  1198.2 ,  1199.82,  1213.93,  1212.88,  1219.18,
        1203.8 ,  1221.71,  1224.75,  1223.81,  1230.07,  1226.29,
        1223.76,  1207.13,  1206.48,  1208.77,  1196.54,  1185.9 ,
        1180.28,  1186.27,  1177.54,  1179.46,  1174.37,  1166.06,
        1166.15,  1182.11,  1187.8 ,  1210.82,  1215.06,  1197.54,
        1184.7 ,  1188.51,  1179.  ,  1190.37,  1202.63,  1193.14,
        1173.53,  1166.51,  1171.39,  1182.66,  1184.58,  1187.31,
        1182.93,  1196.65,  1186.27,  1195.84,  1203.63,  1222.34,
        1223.18,  1224.89,  1215.24,  1227.21,  1196.48,  1185.11,
        1189.19,  1178.32,  1183.43,  1164.78,  1167.67,  1152

In [2]:
data1 = np.loadtxt(open("Data.csv","rb"),delimiter=",",skiprows=0)

In [3]:
X = data1[:,0:9]
y = data1[:,9:10]

In [20]:
theta = np.array([[-0.003055,0.604211,0.321124,-0.022503,-0.047455,-0.037271,0.095452,0.309359,0.087039]])
theta = np.transpose(theta)

In [63]:
theta.shape

(9L, 1L)

In [64]:
print compute_square_loss(X, y, theta)

[[ 0.08348516]]


########################################
###Q2.2b: compute the gradient of square loss function

In [55]:
def compute_square_loss_gradient(X, y, theta):
    """
    Compute gradient of the square loss (as defined in compute_square_loss), at the point theta.
    
    Args:
        X - the feature vector, 2D numpy array of size (num_instances, num_features)
        y - the label vector, 1D numpy array of size (num_instances)
        theta - the parameter vector, 1D numpy array of size (num_features)
    
    Returns:
        grad - gradient vector, 1D numpy array of size (num_features)
    """
    #TODO
    grad = np.dot(np.transpose(theta),np.transpose(X))
    grad = np.dot(grad,X)
    grad = grad - np.dot(np.transpose(y),X)
    grad = grad/(int(y.shape[0]))
    
    return grad    

In [76]:
theta = np.array([[0,0,0,-0,-0,-0,0,0,0]])
theta = np.transpose(theta)

In [83]:
theta = np.array([[-0.003055076,0.604211200,0.321123595,-0.022502569,-0.047454715,-0.037270900,0.095451991,0.309358974,0.087038807]])
theta = np.transpose(theta)

In [84]:
print compute_square_loss_gradient(X, y, theta)

[[ -1.57329400e-04  -4.46819019e-06  -3.00018156e-06  -2.67514120e-06
   -3.90733370e-05  -5.38712805e-06  -7.14089849e-07  -6.06772530e-07
   -2.36076641e-06]]


###########################################
###Q2.3a: Gradient Checker
#Getting the gradient calculation correct is often the trickiest part
#of any gradient-based optimization algorithm.  Fortunately, it's very
#easy to check that the gradient calculation is correct using the
#definition of gradient.
#See http://ufldl.stanford.edu/wiki/index.php/Gradient_checking_and_advanced_optimization


In [56]:
def grad_checker(X, y, theta, epsilon=0.01, tolerance=1e-4): 
    """Implement Gradient Checker
    Check that the function compute_square_loss_gradient returns the
    correct gradient for the given X, y, and theta.

    Let d be the number of features. Here we numerically estimate the
    gradient by approximating the directional derivative in each of
    the d coordinate directions: 
    (e_1 = (1,0,0,...,0), e_2 = (0,1,0,...,0), ..., e_d = (0,...,0,1) 

    The approximation for the directional derivative of J at the point
    theta in the direction e_i is given by: 
    ( J(theta + epsilon * e_i) - J(theta - epsilon * e_i) ) / (2*epsilon).

    We then look at the Euclidean distance between the gradient
    computed using this approximation and the gradient computed by
    compute_square_loss_gradient(X, y, theta).  If the Euclidean
    distance exceeds tolerance, we say the gradient is incorrect.

    Args:
        X - the feature vector, 2D numpy array of size (num_instances, num_features)
        y - the label vector, 1D numpy array of size (num_instances)
        theta - the parameter vector, 1D numpy array of size (num_features)
        epsilon - the epsilon used in approximation
        tolerance - the tolerance error
    
    Return:
        A boolean value indicate whether the gradient is correct or not

    """
    true_gradient = compute_square_loss_gradient(X, y, theta) #the true gradient
    num_features = int(theta.shape[0])
    approx_gradient = np.zeros(num_features) #Initialize the gradient we approximate
    #TODO
    for i in range(0,num_features):

        theta[i] = theta[i] - epsilon
        part1 = compute_square_loss(X, y, theta)
        theta[i] = theta[i] + 2 * epsilon
        part2 = compute_square_loss(X, y, theta)
        approx_gradient[i] = (part2-part1)/(2*epsilon)
        theta[i] = theta[i] - epsilon
        
    for i in range(0,num_features):
        if abs(approx_gradient[i]-true_gradient[0][i])<tolerance:
            continue
        else:
            return False
    return True

In [118]:
grad_checker(X, y, theta, epsilon=0.01, tolerance=1e-4)

True

#################################################
###Q2.3b: Generic Gradient Checker

In [57]:
def generic_gradient_checker(X, y, theta, objective_func, gradient_func, epsilon=0.01, tolerance=1e-4):
    """
    The functions takes objective_func and gradient_func as parameters. And check whether gradient_func(X, y, theta) returned
    the true gradient for objective_func(X, y, theta).
    Eg: In LSR, the objective_func = compute_square_loss, and gradient_func = compute_square_loss_gradient
    """
    #TODO
    
    true_gradient = gradient_func(X, y, theta) #the true gradient
    num_features = int(theta.shape[0])
    approx_gradient = np.zeros(num_features) #Initialize the gradient we approximate
    #TODO
    for i in range(0,num_features):

        theta[i] = theta[i] - epsilon
        part1 = objective_func(X, y, theta)
        theta[i] = theta[i] + 2 * epsilon
        part2 = objective_func(X, y, theta)
        approx_gradient[i] = (part2-part1)/(2*epsilon)
        theta[i] = theta[i] - epsilon

    for i in range(0,num_features):
        if abs(approx_gradient[i]-true_gradient[0][i])<tolerance:
            continue
        else:
            return False
    return True    
    

In [120]:
generic_gradient_checker(X, y, theta, compute_square_loss, compute_square_loss_gradient, epsilon=0.01, tolerance=1e-4)

True

####################################
####Q2.4a: Batch Gradient Descent

In [12]:
train,test = split_train_test(data1)
train_norm, test_norm = feature_normalization(train, test)

In [13]:
X = train_norm[:,0:9]
b = np.expand_dims(np.ones(int(X.shape[0])), axis=1)
X = np.hstack((X,b))
y = train_norm[:,9:10]

In [58]:
def batch_grad_descent(X, y, theta, alpha=0.1, num_iter=1000, check_gradient=False):
    """
    In this question you will implement batch gradient descent to
    minimize the square loss objective
    
    Args:
        X - the feature vector, 2D numpy array of size (num_instances, num_features)
        y - the label vector, 1D numpy array of size (num_instances)
        alpha - step size in gradient descent
        num_iter - number of iterations to run 
        check_gradient - a boolean value indicating whether checking the gradient when updating
        
    Returns:
        theta_hist - store the the history of parameter vector in iteration, 2D numpy array of size (num_iter+1, num_features) 
                    for instance, theta in iteration 0 should be theta_hist[0], theta in ieration (num_iter) is theta_hist[-1]
        loss_hist - the history of objective function vector, 1D numpy array of size (num_iter+1) 
    """
    num_instances, num_features = int(X.shape[0]), int(X.shape[1])
    theta_hist = np.zeros((num_iter+1, num_features))  #Initialize theta_hist
    loss_hist = np.zeros(num_iter+1) #initialize loss_hist
    theta = np.expand_dims(np.ones(num_features), axis=1) #initialize theta
    #TODO
    
    if check_gradient==True:
        c = generic_gradient_checker(X, y, theta, compute_square_loss, compute_square_loss_gradient, epsilon=0.01, tolerance=1e-4)
        if c == True:
            for i in range(0,num_iter+1):    
                loss_hist[i] =  loss_hist[i]+compute_square_loss(X, y, theta)[0]
                theta_hist[i] = theta_hist[i]+np.transpose(theta)[0]
                g = compute_square_loss_gradient(X, y, theta) 
                if i >3 :
                    if sum(abs(theta_hist[i]-theta_hist[i-2]))/num_features<1e-3:
                        print ("iteration: %s times" % (i))
                        return (theta_hist,loss_hist)               
                
                g[0]= g[0]/math.sqrt(sum(np.multiply(g[0],g[0])))
                theta = theta-np.transpose(g)*alpha

            return (theta_hist,loss_hist)
        else:
            return (None,None)
    else:
        for i in range(0,num_iter+1):    
            loss_hist[i] =  loss_hist[i]+compute_square_loss(X, y, theta)[0]
            theta_hist[i] = theta_hist[i]+np.transpose(theta)[0]
            g = compute_square_loss_gradient(X, y, theta)
            if i >3 :
                if sum(abs(theta_hist[i]-theta_hist[i-2]))/num_features<1e-3:
                    print ("iteration: %s times" % (i))
                    return (theta_hist,loss_hist)  
            g[0]= g[0]/math.sqrt(sum(np.multiply(g[0],g[0])))
            theta = theta-np.transpose(g)*alpha

        return (theta_hist,loss_hist)

In [35]:
h_theta,h_l=batch_grad_descent(X, y, alpha=0.01, num_iter=1000, check_gradient=False)

iteration: 857 times


In [248]:
h_theta[-1]

array([  2.69997803e-04,   3.68835239e-01,   2.73713723e-01,
         1.60426581e-01,  -1.96103840e-01,   1.07945297e-02,
         8.54045841e-03,   9.17103811e-02,   1.55418216e-01,
        -1.85271272e-02])

In [159]:
plt.plot(h_l[400:])
plt.show()

####################################
###Q2.4b: Implement backtracking line search in batch_gradient_descent
###Check http://en.wikipedia.org/wiki/Backtracking_line_search for details
#TODO

In [59]:
def backtracking_line_search(X,y,theta,p,c=0.5,tao=0.5,num_iter=1000,alpha=10):
    t = np.dot(p,compute_square_loss_gradient(X, y, theta)[0])[0]
    t = -t * c
    
    for j in range(0,num_iter):
        if (compute_square_loss(X, y, theta)-compute_square_loss(X, y, (theta+alpha*np.transpose(p))))>alpha*t:
            return alpha
        else:
            alpha=alpha*tao
    print "not finding within iter"
    return alpha

In [259]:
theta = np.expand_dims(h_theta[-1], axis=1) #initialize theta
p=compute_square_loss_gradient(X, y, theta)

In [60]:
def backtracking_further(X,y,theta,p,num_iter=10000):
    for i in range(0,num_iter):
        al = backtracking_line_search(X,y,theta,p,c=0.5,tao=0.5,num_iter=1000,alpha=1e-9)
        theta = theta-alpha*np.transpose(p)
        p=compute_square_loss_gradient(X, y, theta)
    return theta

In [268]:
jj = backtracking_further(X,y,theta,p,num_iter=1000)

###################################################
###Q2.5a: Compute the gradient of Regularized Batch Gradient Descent


In [61]:
def compute_regularized_square_loss_gradient(X, y, theta, lambda_reg):
    """
    Compute the gradient of L2-regularized square loss function given X, y and theta
    
    Args:
        X - the feature vector, 2D numpy array of size (num_instances, num_features)
        y - the label vector, 1D numpy array of size (num_instances)
        theta - the parameter vector, 1D numpy array of size (num_features)
        lambda_reg - the regularization coefficient
    
    Returns:
        grad - gradient vector, 1D numpy array of size (num_features)
    """
    #TODO
    grad = np.dot(np.transpose(theta),np.transpose(X))
    grad = np.dot(grad,X)
    grad = grad - np.dot(np.transpose(y),X)
    grad = grad/(int(y.shape[0]))
    grad = grad + 2 * lambda_reg * np.transpose(theta)
    
    return grad    

In [24]:
theta = np.expand_dims(np.ones(X.shape[1]), axis=1) #initialize theta

In [ ]:
X = train_norm[:,0:9]
b = np.expand_dims(np.ones(int(X.shape[0])), axis=1)
X = np.hstack((X,b))
y = train_norm[:,9:10]

In [26]:
compute_regularized_square_loss_gradient(X, y, theta, lambda_reg=1)

array([[ 4.97523073,  4.9562603 ,  4.59638379,  4.72663646,  4.38927439,
         4.72905532,  3.8421146 ,  4.3638974 ,  4.8588193 ,  6.85673934]])

###################################################
###Q2.5b: Batch Gradient Descent with regularization term

In [62]:
def compute_regularized_square_loss(X, y, theta, lambda_reg):
    """
    Given a set of X, y, theta, compute the square loss for predicting y with X*theta
    
    Args:
        X - the feature vector, 2D numpy array of size (num_instances, num_features)
        y - the label vector, 1D numpy array of size (num_instances)
        theta - the parameter vector, 1D array of size (num_features)
    
    Returns:
        loss - the square loss, scalar
    """
    loss = 0 #initialize the square_loss
    #TODO
    
    part = np.dot(X,theta)-y
    loss = loss + np.dot(np.transpose(part),part)
    loss = loss/(2*int(y.shape[0]))
    loss = loss + lambda_reg *  np.dot(np.transpose(theta),theta)[0]
    
    return loss

In [63]:
def regularized_grad_descent(X, y, theta, alpha=0.1, lambda_reg=1, num_iter=1000):
    """
    Args:
        X - the feature vector, 2D numpy array of size (num_instances, num_features)
        y - the label vector, 1D numpy array of size (num_instances)
        alpha - step size in gradient descent
        lambda_reg - the regularization coefficient
        numIter - number of iterations to run 
        
    Returns:
        theta_hist - the history of parameter vector, 2D numpy array of size (num_iter+1, num_features) 
        loss_hist - the history of regularized loss value, 1D numpy array
    """
    num_instances, num_features = int(X.shape[0]), int(X.shape[1])
    theta_hist = np.zeros((num_iter+1, num_features))  #Initialize theta_hist
    loss_hist = np.zeros(num_iter+1) #initialize loss_hist
    theta = np.expand_dims(np.ones(num_features), axis=1) #initialize theta
    #TODO
    for i in range(0,num_iter+1):    
        loss_hist[i] =  loss_hist[i]+compute_regularized_square_loss(X, y, theta, lambda_reg)[0]
        theta_hist[i] = theta_hist[i]+np.transpose(theta)[0]
        g = compute_regularized_square_loss_gradient(X, y, theta, lambda_reg)
        g[0]= g[0]/math.sqrt(sum(np.multiply(g[0],g[0])))
        theta = theta-np.transpose(g)*alpha

    return (theta_hist,loss_hist)

In [78]:
a,b= regularized_grad_descent(X, y, alpha=0.1, lambda_reg=1, num_iter=1000)

In [80]:
a[1]

array([ 0.97974482,  0.97998011,  0.98020227,  0.98049959,  0.98127342,
        0.98187444,  0.98258692,  0.98291303,  0.98404658,  0.98592979,
        0.98733536,  0.98809273,  0.98974305,  0.99152843,  0.99347988,
        0.99382115,  0.99527347,  0.99756935,  0.98219701,  0.98219701,
        0.98219701,  0.98365787,  0.98365787,  0.98365787,  0.98485586,
        0.98485586,  0.98485586,  0.98540977,  0.98540977,  0.98540977,
        0.98571674,  0.98571674,  0.98571674,  0.99106   ,  0.99106   ,
        0.99106   ,  0.98909769,  0.98909769,  0.98909769,  0.98796722,
        0.98796722,  0.98796722,  0.98745262,  0.98745262,  0.98745262,
        0.98716986,  0.98716986,  0.98716986,  0.97964921])

In [66]:
Xt = test_norm[:,0:48]
b = np.expand_dims(np.ones(int(Xt.shape[0])), axis=1)
Xt = np.hstack((Xt,b))
yt = test_norm[:,48:49]

In [1]:
def find_minimizer_lambda(X,y,Xt,yt,theta, lambda_start_log = -7.0,lambda_end_log = 3.0,num_iter=10):
    for i in range(0,10):
        loss={"train":[],"valid":[]}
        r = (lambda_end_log-lambda_start_log)/10
        lambda_range = []
        plots = []
        for i in range(0,num_iter):
            lambda_range.append(math.exp(math.log(10)*(lambda_start_log+r*i)))
        for i in lambda_range:
            a,b=regularized_grad_descent(X, y,theta,  alpha=0.1, lambda_reg=i, num_iter=1000)
            loss["train"].append(compute_regularized_square_loss(X, y, np.transpose(a[-2:-1]),i)[0])
            loss["valid"].append(compute_square_loss(Xt, yt, np.transpose(a[-2:-1]))[0])

        minl = loss["valid"].index(min(loss["valid"]))

        if minl == 0:
            lambda_end_log = lambda_start_log+r
        elif minl == len(lambda_range)-1:
            lambda_start_log = lambda_end_log-r
        else:
            if loss["valid"][minl-1]<loss["valid"][minl+1]:
                lambda_start_log = lambda_start_log + (minl-1)*r
                lambda_end_log = lambda_start_log + r
            else:
                lambda_start_log = lambda_start_log + (minl)*r
                lambda_end_log = lambda_start_log + r

    return lambda_range[loss["valid"].index(min(loss["valid"]))]


In [68]:
best_lambda = find_minimizer_lambda(lambda_start_log = -7.0,lambda_end_log = 3.0,num_iter=10)

(Optional) Once you have found a good value for λ, repeat the fits with different values for
B, and plot the results. For this dataset, does regularizing the bias help, hurt, or make no
significant difference?

In [155]:
loss_r = []
loss_b = []
for i in range(-5,5):
    X = train_norm[:,0:9]
    b = np.expand_dims(np.ones(int(X.shape[0])), axis=1) * math.pow(10,i)
    X = np.hstack((X,b))
    y = train_norm[:,9:10]
    h_theta,h_loss= regularized_grad_descent(X, y, alpha=0.1, lambda_reg=best_lambda, num_iter=1000)
    h_theta1,h_loss1= batch_grad_descent(X, y, alpha=0.1, num_iter=1000, check_gradient=False)
    loss_r.append(h_loss[-1])
    loss_b.append(h_loss1[-1])

In [158]:
plt.plot(loss_r,loss_b)
plt.xticks(range(10), range(-5,5))
plt.show()

(Optional) Estimate the average time it takes on your computer to compute a single gradient
step.

In [171]:
n=1000
start_time = time.time()
regularized_grad_descent(X, y, alpha=0.1, lambda_reg=best_lambda, num_iter=n)
elasp = (time.time() - start_time)
print("--- %s seconds per gradient---" % (elasp/n) )


--- 4.99999523163e-05 seconds per gradient---


#############################################
###Q2.6a: Stochastic Gradient Descent

In [15]:
X = train_norm[:,0:48]
b = np.expand_dims(np.ones(int(X.shape[0])), axis=1)
X = np.hstack((X,b))
y = train_norm[:,48:49]

In [1]:
def stochastic_grad_descent(train,loss_f,grad_f, alpha, lambda_reg, reshuffling=False, num_iter=1000,alpha_sqrt=False,B=1):
    """
    In this question you will implement stochastic gradient descent with a regularization term
    
    Args:
        X - the feature vector, 2D numpy array of size (num_instances, num_features)
        y - the label vector, 1D numpy array of size (num_instances)
        alpha - string or float. step size in gradient descent
                NOTE: In SGD, it's not always a good idea to use a fixed step size. Usually it's set to 1/sqrt(t) or 1/t
                if alpha is a float, then the step size in every iteration is alpha.
                if alpha == "1/sqrt(t)", alpha = 1/sqrt(t)
                if alpha == "1/t", alpha = 1/t
        lambda_reg - the regularization coefficient
        num_iter - number of epochs (i.e number of times) to go through the whole training set
    
    Returns:
        theta_hist - the history of parameter vector, 3D numpy array of size (num_iter, num_instances, num_features) 
        loss hist - the history of regularized loss function vector, 2D numpy array of size(num_iter, num_instances)
    """
        
    train1 = train
    X = train1[:,0:48]
    b = np.expand_dims(np.ones(int(X.shape[0])), axis=1) *B 
    X = np.hstack((X,b))
    y = train1[:,48:49]
        
    num_instances, num_features = int(X.shape[0]), int(X.shape[1])
    theta = np.expand_dims(np.ones(num_features), axis=1) #initialize theta
    g_lag = 0
    theta_hist = np.zeros((num_iter, num_instances, num_features))  #Initialize theta_hist
    loss_hist = np.zeros((num_iter, num_instances)) #Initialize loss_hist
    
    alpha =  alpha / (1+ alpha *lambda_reg)
    
    for i in range(0,num_iter):  
        #if alpha_sqrt==False: alpha=1.0/(i+1)
        #else: alpha=1.0/math.sqrt(i+1)
        for j in range(0,num_instances):
            loss_hist[i][j] =  loss_hist[i][j]+loss_f(X[j:j+1], y[j:j+1], theta, lambda_reg)[0]
            theta_hist[i][j] = theta_hist[i][j]+np.transpose(theta)[0]
            g = grad_f(X[j:j+1], y[j:j+1], theta, lambda_reg)
            if j > 3:
                if sum(np.transpose(abs(g+g_lag))*alpha)/num_features<1e-5:
                    print ("iteration: %s times; now on %sth instance" % (i,j))
                    return (theta_hist,loss_hist)
            g[0]= g[0]/math.sqrt(sum(np.multiply(g[0],g[0])))
            theta = theta-np.transpose(g)*alpha
            g_lag = g

        if reshuffling==True:
            np.random.shuffle(train1)
            X = train1[:,0:48]
            b = np.expand_dims(np.ones(int(X.shape[0])), axis=1)*B 
            X = np.hstack((X,b))
            y = train1[:,48:49]
    return (theta_hist,loss_hist)
    #TODO

In [87]:
start_time = time.time()
a,b=stochastic_grad_descent(train_norm, alpha=0.1, lambda_reg=best_lambda,reshuffling=False, num_iter=10000,alpha_sqrt=False)
print("--- %s seconds ---" % (time.time() - start_time))

iteration: 1383 times; now on 58th instance
--- 13.0529999733 seconds ---


In [62]:
start_time = time.time()
a,b=stochastic_grad_descent(train_norm, alpha=0.1, lambda_reg=best_lambda,reshuffling=True, num_iter=10000,alpha_sqrt=False)
print("--- %s seconds ---" % (time.time() - start_time))

--- 92.6349999905 seconds ---


In [37]:
start_time = time.time()
a,b=batch_grad_descent(X, y, alpha=0.1, num_iter=1000, check_gradient=False)
print("--- %s seconds ---" % (time.time() - start_time))

iteration: 114 times
--- 0.0139999389648 seconds ---


In [94]:
a,b=stochastic_grad_descent(train, compute_regularized_square_loss,compute_regularized_square_loss_gradient,alpha=0.1 , lambda_reg=best_lambda,reshuffling=False, num_iter=1000,alpha_sqrt=False,B=10)

iteration: 71 times; now on 70th instance


In [91]:
a[72][70]

array([ 0.83316732,  1.50220708,  1.08556428,  0.40782899,  0.24043455,
       -0.21588492,  0.32424505,  0.5386682 ,  1.73562373,  0.64277362,
        0.53705342, -0.2034212 , -0.21727123, -0.18997313,  1.1980233 ,
        0.97219928,  0.88458695,  1.27520022,  0.7156433 ,  0.7156433 ,
       -0.4217781 ,  0.63616279,  0.63616279, -0.81918067,  0.64641752,
        0.64641752, -0.76790703,  0.67546741,  0.67546741, -0.62265755,
        0.70591098,  0.70591098, -0.47043971,  0.70044703,  0.70044703,
       -0.49775945,  0.73042004,  0.73042004, -0.34789443,  0.76794635,
        0.76794635, -0.16026287,  0.79929185,  0.79929185, -0.00353535,
        0.82420764,  0.82420764,  0.12104358,  0.82838745])

In [96]:
a,b=stochastic_grad_descent(train, compute_regularized_square_loss,compute_regularized_square_loss_gradient,alpha=0.1 , lambda_reg=best_lambda,reshuffling=False, num_iter=1000,alpha_sqrt=False,B=10)

In [97]:
a,b=stochastic_grad_descent(train, compute_regularized_square_loss,compute_regularized_square_loss_gradient,alpha=0.05 , lambda_reg=best_lambda,reshuffling=False, num_iter=1000,alpha_sqrt=False,B=10)

iteration: 440 times; now on 23th instance


In [98]:
a,b=stochastic_grad_descent(train, compute_regularized_square_loss,compute_regularized_square_loss_gradient,alpha=0.005 , lambda_reg=best_lambda,reshuffling=False, num_iter=1000,alpha_sqrt=False,B=10)

iteration: 61 times; now on 52th instance


Try a stepsize rule of the form ηt = 1+η η0 0λt, where λ is your regularization constant,
and η0 a constant you can choose. How do the results compare?

In [100]:
a,b=stochastic_grad_descent(train, compute_regularized_square_loss,compute_regularized_square_loss_gradient,alpha=0.1 , lambda_reg=best_lambda,reshuffling=False, num_iter=1000,alpha_sqrt=False,B=10)

In [101]:
a,b=stochastic_grad_descent(train, compute_regularized_square_loss,compute_regularized_square_loss_gradient,alpha=0.05 , lambda_reg=best_lambda,reshuffling=False, num_iter=1000,alpha_sqrt=False,B=10)

iteration: 440 times; now on 23th instance


In [102]:
a,b=stochastic_grad_descent(train, compute_regularized_square_loss,compute_regularized_square_loss_gradient,alpha=0.005 , lambda_reg=best_lambda,reshuffling=False, num_iter=1000,alpha_sqrt=False,B=10)

iteration: 61 times; now on 52th instance
